In [1]:
from lp_coco_utils.lp_getDataset import getDatasetProcessed
from lp_training.lp_trainer import train
from lp_training.lp_loss import computeLoss
from lp_model.lp_litepose import LitePose
import lp_config.lp_common_config as cc
import torch
from lp_training.lp_inference import inference
from lp_utils.lp_image_processing import drawHeatmap, drawKeypoints, normalizeImage
from lp_testing.lp_evaluation import computeOKS

This file has to be seen only as an entry that calls wrapper functions, the implentation of those functions can be found in the subdirectories of the repository.   
Every hyperparameter can be edited in `src/lp_config`.  
`lp_common_config.py` holds the general configurations about the dataset loading, training and test. On the other hand `lp_model_config.py` contrains the parameters that encode the model structure. The current model configs are taken from the Neural Architecture Search performed by the paper authors. I took the small size network due to the computational power available, however better results can be achieved simply by scaling the network size (Good parameters combinations are provided by the paper authors).

Code taken by the [official paper repository](https://github.com/mit-han-lab/litepose):
- classes `CocoDataset` and `CocoKeypoints` are partially taken, I added fiftyone support that makes the dataset setup easier and I removed unnecessary code.
- I took the code inside `lp_generators.py` and `lp_transforms.py` as well, since they were a `CocoKeypoints` dependencies

# Training

The dataset is downloaded by using fiftyone APIs and keypoint heatmaps are created for each sample. 

In [2]:
train(cc.config["batch_size"])

Found annotations at '/home/daniaffch/fiftyone/coco-2017/raw/instances_train2017.json'
Images already downloaded
Existing download of split 'train' is sufficient
loading annotations into memory...
Done (t=4.47s)
creating index...
index created!
Found annotations at '/home/daniaffch/fiftyone/coco-2017/raw/instances_val2017.json'
Images already downloaded
Existing download of split 'validation' is sufficient
loading annotations into memory...
Done (t=0.71s)
creating index...
index created!


  0%|          | 24/8015 [00:04<26:42,  4.99it/s] 


KeyboardInterrupt: 

# Inference
Unfotunately OpenCV method `imshow()` has a well known bug with python notebooks.

In [3]:
import cv2

model = LitePose().to(cc.config["device"])
model.load_state_dict(torch.load("definitiveTag"))

ds = getDatasetProcessed("validation")

data_loader = torch.utils.data.DataLoader(
    ds,
    batch_size=8
)

row = next(iter(data_loader))
images = row[0].to(cc.config["device"])
gthm = row[1]
output, keypoints = inference(model, images)

Found annotations at '/home/daniaffch/fiftyone/coco-2017/raw/instances_val2017.json'
Images already downloaded
Existing download of split 'validation' is sufficient
loading annotations into memory...
Done (t=0.15s)
creating index...
index created!


In [11]:
print(output[1][2][:cc.config["num_joints"]].shape)

torch.Size([17, 128, 128])


In [4]:
import numpy as np

images = row[0].to(cc.config["device"])
gthm = row[1]
gtmask = row[2]
gtj = row[3]
heatmaps = [h.to(cc.config["device"]) for h in gthm]
masks = [h.to(cc.config["device"]) for h in gtmask]
joints = [j.to(cc.config["device"]) for j in gtj]

for n,h in enumerate(output[1][2][:cc.config["num_joints"]]):
    h = normalizeImage(h).cpu()
    finalHm = cv2.applyColorMap(np.uint8(h), cv2.COLORMAP_JET)

    hgt = normalizeImage(gthm[1][2][n])
    finalHmgt = cv2.applyColorMap(np.uint8(hgt), cv2.COLORMAP_JET)

    loss = (h - hgt)**2 * gtmask[1][2][n]
    loss = normalizeImage(loss)
    loss = cv2.applyColorMap(np.uint8(loss), cv2.COLORMAP_JET)

    cv2.imshow("Out"+str(n), finalHm)
    cv2.imshow("Gt"+str(n), finalHmgt)
    cv2.imshow("Loss"+str(n), loss)

    cv2.waitKey()

computeLoss(output, heatmaps, joints)

TypeError: computeLoss() missing 1 required positional argument: 'gtJoints'

In [13]:
jointsHeatmap = output[1][2][:cc.config["num_joints"]]

img, finalHm, superimposed = drawHeatmap(images[2], jointsHeatmap)
img, gtfinalHm, gtsuperimposed = drawHeatmap(images[2], gthm[1][2])
cv2.imshow("Image", img)
cv2.imshow("Final heatmap", finalHm)
cv2.imshow("Superimposed", superimposed)

cv2.imshow("Ground Truth heatmap", gtfinalHm)
cv2.imshow("Ground Truth Superimposed", gtsuperimposed)
cv2.waitKey()

113

In [3]:
img = drawKeypoints(images[0], keypoints[0])
cv2.imshow("Image Keypoints", img)
cv2.waitKey()

113